# Chapter 02

## Class Definitions
In python, every data is an instance of some class.

### Example: Credit Card Class

In [54]:
print(list().__class__.__name__)
print(str().__class__.__name__)
print(dict().__class__.__name__)

list
str
dict


In [55]:
print(list().__class__.__class__.__name__)

type


The `self` identifier explicitly identifies the instance that a method is invoked in class.


In [56]:
class CreditCard:
    """A consumer credit card"""
    
    def __init__(self, customer, bank, acnt, limit):
        """Create a new credit card instance.
        
        The initial balance is zero.
        
        :param customer: the name of the customer 
        :param bank: the name of the bank
        :param acnt: the account identifier
        :param limit: credit limit
        """
        
        self._customer = customer
        self._bank = bank
        self._account = acnt
        self._limit = limit
        self._balance = 0
        
    def get_customer(self):
        """Return name of the customer"""
        return self._customer
    
    def get_bank(self):
        """Return the bank's name."""
        return self._bank
    
    def get_account(self):
        """Return the card identifying number (typically stored as a string."""
        return self._account
    
    def get_limit(self):
        """Return current credit limit."""
        return self._limit
    
    def get_balance(self):
        """Return current balance."""
        return self._balance
    
    def charge(self, price):
        """Charge given price to the card, assuming sufficient credit limit.
        
        Return True if charge was processed; False if charge was denied.
        """
        
        if price + self._balance > self._limit:
            return False
        else:
            self._balance += price
            return True
    
    def make_payment(self, amount):
        """Process customer payment that reduces balance."""
        self._balance -= amount 

Python interpreter automatically binds the instance upon which the method is invoked to the `self` parameter.


In [57]:
cc = CreditCard('John Doe', 'Bank', '1234 5678', 1000)
cc

In class, `__init__` method works as the **constructor** of the class. Also, single leading underscore in the name of a data member, such as `_balance`, implies that it is intended as **nonpublic**. Users of a class should not directly access such members.

For better encapsulation, it is mostly better to treat all data members as nonpublic and provide accessors, to provide a user of our class read-only access to a trait, and update methods for updating its memebers.

### Operator overloading and Python's Special Methods
By default, operators can not work on classes unless special methods are defined. You can see detailed info at [here](https://docs.python.org/3/reference/datamodel.html#special-method-names). Python also supports non-operator overloads. For example, `str` invokes `__str__()` and `bool` invokes `__bool__()`. However, we should not carelessly assume that Python will manage all the implications. Defining `__eq__` will support syntax `a == b`, but it does not affect the evaluation of a syntax `a != b`, which should be defined by `__ne__`. In similar context, defining `__eq__` and `__lt__` does not imply semantics for a `a <= b `.

### Example: Multidimensional Vector Class

In [58]:
class Vector:
    """Represent a vector in a multidimensional space"""
    
    def __init__(self, d):
        """Create d-dimensional vector of zeros."""
        self._coords = [0] * d
        
    def __len__(self):
        """Return the dimension of the vector."""
        return len(self._coords)
    
    def __getitem__(self, j):
        """Return jth coordinate of vector."""
        return self._coords[j]
    
    def __setitem__(self, j, val):
        """Set jth coordinate of vector to given value."""
        self._coords[j] = val
    
    def __add__(self, other):
        """Return sum of two vectors."""
        if len(self) != len(other):
            raise ValueError('dimensions must agree')
        result = Vector(len(self))
        for j in range(len(self)):
            result[j] = self[j] + other[j]
        return result
    
    def __eq__(self, other):
        """Return True if vectgor has same coordinates as other."""
        return self._coords == other._coords
    
    def __ne__(self, other):
        """Return True if vector differs from other."""
        return not self == other  # This rely on existing __eq__ definition
    
    def __str__(self):
        """Produce string representation of vector."""
        return '<' + str(self._coords)[1:-1] + '>'
    
    def __repr__(self):
        """For representation."""
        return '<' + str(self._coords)[1:-1] + '>'

In [59]:
x = Vector(5)
x[2] = 10
x

<0, 0, 10, 0, 0>

In [60]:
x + x

<0, 0, 20, 0, 0>

In [61]:
x * x  # Gives error since it is not defined through `__mul__`

TypeError: unsupported operand type(s) for *: 'Vector' and 'Vector'

### Iterators
An **Iterator** for a collection provides one key behavior: It supports a special method named `__next__` that returns the next element of the collection, if any, or raises a `StopIteration` exception to indicate that there are no further elements.

Python automatically supports iterator implementation if both `__len__` and `__getitem__` is defined.

In [ ]:
class SequenceIterator:
    """An iterator for any of Python's sequence types."""
    
    def __init__(self, sequence):
        """Create an iterator for the given sequence."""
        self._seq = sequence  # keep a reference to the underlying data
        self._k = -1  # will increment to 0 on first call to next
    
    def __next__(self):
        """Return the next element, or else raise StopIteration error."""
        self._k += 1
        if self._k < len(self._seq):
            return(self._seq[self._k])
        else:
            raise StopIteration()
        
    def __iter__(self):
        """By convention, an iterator must return itself as an iterator."""
        return self

In [62]:
custom_iter = SequenceIterator([4, 2, 1])
iter(custom_iter)

In [63]:
for i in SequenceIterator([5, 2, 1, 2, 7]):
    print(i)

5
2
1
2
7


### Example: Range Class
This example mimics Pthon's built-in `range` class. There is a big difference between Python 3's `range` and Python 2's range. Basically, Python 3's `range` works like a Python 2's `xrange`.

The core difference between them is that Python 3's range employs **lazy evaluation**, which does not create a new list instance, which might be expensive if the list is huge, effectively repesenting the desired range of elements without storing them in memory.

In [64]:
class Range:
    """A class that mimic's the built-in range class."""
    
    def __init__(self, start, stop=None, step=1):
        """Initialize a Range Instance
        Semantics is similar to built-in range class
        """
        if step == 0:
            raise ValueError('step cannot be 0')
        
        if stop is None:
            start, stop = 0, start
            
        # calculate the effective length once
        self._length = max(0, (stop - start + step -1) // step)
        
        # nned knowledge of start and step (but not step) to support _-getitem__
        self._start = start
        self._step = step
        
    def __len__(self):
        """Return number of entries in the range."""
        return self._length
    
    def __getitem__(self, k):
        """Return entry at index k (using standard interpretation if negative.)"""
        if k < 0:
            k += len(self)
        
        if not 0 <= k < self._length:
            raise IndexError('index out of range')
        
        return self._start + k * self._step

## Inheritance
In object-oriented programming, the mechanism for a modular and hierarchical organization is a technique known as **inheritance**. This allows a new class to be defined based upon an existing class as the starting point. In object-oriented terminology, the existing class is typically described as the **base class**, **parent class**, or **superclass**, while the newly defined classs is known as the **subclass** or **child class**.

### Extending the CreditCard Class

In [70]:
class PredatoryCreditCard(CreditCard):
    """An extension to CreditCard that compounds interest and fees."""
    
    def __init__(self, customer, bank, acnt, limit, apr):
        """Create a new predatory credit card instance.
        
        The initial balance is zero.
        customer    the name of the customer
        bank        the name of the bank
        acnt        the acount identifier
        limit       credit limit
        apr         annual percentage rate
        """

        super().__init__(customer, bank, acnt, limit)
        self._apr = apr
        
    def charge(self, price):
        """Charge given price to the card, assuming sufficient credit limit.
        
        Return True if charge was processed.
        Return False and assess $5 fee if charge is denied.
        """
        
        success = super().charge(price)
        if not success:
            self._balance += 5
        return success
    
    def process_month(self):
        """Assess monthly interest on outstanding balance."""
        if self._balance > 0:
            # if positive balance, convert APR to monthly multiplicative factor
            monthly_factor = pow(1 + self._apr, 1/12)
            self._balance *= monthly_factor

In [77]:
predatory = PredatoryCreditCard('John', 'AA Bank', '1234 5678', 100000000, 0.03)
predatory.charge(1000)
predatory.get_balance()

1000

In [78]:
predatory.process_month()
predatory.get_balance()

1002.4662697723037

#### Protected Members
In Python, names beginning with a single underscore are conventionally akin to **protected**, which are accessible to subclasses, but not to the general public, while names beginning with a double underscore (other than special methods) are akin to **private**, which are not accessible to both subclasses and general public.

However, this access control is not supported formally by Python. Keep in mind that it is just a convention. So if you are trying to use some attributes followed by an underscore, think twice that it could compromise the class's designer's intention.

### Hierarchy of Numeric Progresions
